In [12]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0, Xception
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [13]:
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 10
SEED = 42



In [14]:
original_df = pd.read_csv("ALZ_working\labels.csv")
eff_kmeans_df = pd.read_csv("EfficientNetB0_KMeans.csv")

# If you have Xception-Birch file:
xcep_birch_df = pd.read_csv("Xception_Birch.csv")

In [15]:
def prepare_data(df, label_column):

    df = df.copy()

    le = LabelEncoder()
    df["label_encoded"] = le.fit_transform(df[label_column])

    num_classes = df["label_encoded"].nunique()

    X = df["image_name"].values
    y = to_categorical(df["label_encoded"], num_classes)

    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y,
        test_size=0.3,
        random_state=SEED,
        stratify=df["label_encoded"]
    )

    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp,
        test_size=0.5,
        random_state=SEED
    )

    return X_train, X_val, X_test, y_train, y_val, y_test, num_classes


In [16]:
import os

MIXED_FOLDER = "ALZ_working\mixed_images"   

def load_image(image_name):
    full_path = os.path.join(MIXED_FOLDER, image_name)

    img = tf.keras.preprocessing.image.load_img(
        full_path,
        target_size=(IMG_SIZE, IMG_SIZE)
    )
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = img / 255.0
    return img


def create_dataset(X, y):
    images = np.array([load_image(name) for name in X])
    return images, y


In [17]:
def build_efficientnet(num_classes):

    base = EfficientNetB0(
        weights=None,            # NO pretrained
        include_top=False,
        input_shape=(IMG_SIZE, IMG_SIZE, 3)
    )

    x = GlobalAveragePooling2D()(base.output)
    output = Dense(num_classes, activation="softmax")(x)

    model = Model(inputs=base.input, outputs=output)

    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )

    return model

In [18]:
def build_xception(num_classes):

    base = Xception(
        weights=None,           # NO pretrained
        include_top=False,
        input_shape=(IMG_SIZE, IMG_SIZE, 3)
    )

    x = GlobalAveragePooling2D()(base.output)
    output = Dense(num_classes, activation="softmax")(x)

    model = Model(inputs=base.input, outputs=output)

    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )

    return model

In [19]:
print(original_df.columns)


Index(['image_name', 'label'], dtype='object')


In [20]:
X_train, X_val, X_test, y_train, y_val, y_test, num_classes = prepare_data(original_df,"label")

X_train_img, y_train = create_dataset(X_train, y_train)
X_val_img, y_val = create_dataset(X_val, y_val)
X_test_img, y_test = create_dataset(X_test, y_test)

model_eff_original = build_efficientnet(num_classes)

model_eff_original.fit(
    X_train_img, y_train,
    validation_data=(X_val_img, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE
)

loss1, acc1 = model_eff_original.evaluate(X_test_img, y_test)
print("EfficientNet trained on ORIGINAL labels Accuracy:", acc1)

Epoch 1/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 208s 1s/step - accuracy: 0.5098 - loss: 1.0466 - val_accuracy: 0.3646 - val_loss: 1.1060
Epoch 2/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 179s 1s/step - accuracy: 0.5864 - loss: 0.8550 - val_accuracy: 0.4823 - val_loss: 3.2254
Epoch 3/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 179s 1s/step - accuracy: 0.6922 - loss: 0.6975 - val_accuracy: 0.4823 - val_loss: 3.2425
Epoch 4/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 180s 1s/step - accuracy: 0.7980 - loss: 0.5058 - val_accuracy: 0.4823 - val_loss: 28.5395
Epoch 5/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 178s 1s/step - accuracy: 0.8632 - loss: 0.3483 - val_accuracy: 0.5094 - val_loss: 2.9594
Epoch 6/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 179s 1s/step - accuracy: 0.9098 - loss: 0.2316 - val_accuracy: 0.4969 - val_loss: 4.4960
Epoch 7/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 180s 1s/step - accuracy: 0.9279 - loss: 0.1956 - val_accuracy: 0.7521 - val_loss: 1.1528
Epoch 8/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 179s 1s/step - accuracy: 0.9449 - loss: 0.1565 - val_acc

In [21]:
X_train, X_val, X_test, y_train, y_val, y_test, num_classes = prepare_data(
    eff_kmeans_df,
    label_column="cluster_label"   # <-- adjust if needed
)

X_train_img, y_train = create_dataset(X_train, y_train)
X_val_img, y_val = create_dataset(X_val, y_val)
X_test_img, y_test = create_dataset(X_test, y_test)




model_eff_cluster = build_efficientnet(num_classes)

model_eff_cluster.fit(
    X_train_img, y_train,
    validation_data=(X_val_img, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE
)

loss2, acc2 = model_eff_cluster.evaluate(X_test_img, y_test)

print("\nEfficientNet trained on KMeans labels Accuracy:", acc2)


Epoch 1/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 208s 1s/step - accuracy: 0.8306 - loss: 0.4716 - val_accuracy: 0.8229 - val_loss: 1.0512
Epoch 2/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 179s 1s/step - accuracy: 0.8935 - loss: 0.2417 - val_accuracy: 0.8229 - val_loss: 1.3572
Epoch 3/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 181s 1s/step - accuracy: 0.9141 - loss: 0.1916 - val_accuracy: 0.8229 - val_loss: 1.3543
Epoch 4/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 178s 1s/step - accuracy: 0.9400 - loss: 0.1417 - val_accuracy: 0.8229 - val_loss: 2.9152
Epoch 5/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 167s 1s/step - accuracy: 0.9413 - loss: 0.1438 - val_accuracy: 0.8510 - val_loss: 0.5014
Epoch 6/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 170s 1s/step - accuracy: 0.9444 - loss: 0.1264 - val_accuracy: 0.9021 - val_loss: 0.2756
Epoch 7/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 168s 1s/step - accuracy: 0.9580 - loss: 0.1003 - val_accuracy: 0.9302 - val_loss: 0.1858
Epoch 8/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 168s 1s/step - accuracy: 0.9681 - loss: 0.0793 - val_accu

In [22]:
# Prepare data (original labels)

X_train, X_val, X_test, y_train, y_val, y_test, num_classes = prepare_data(
    original_df,
    label_column="label"   # original label column
)

X_train_img, y_train = create_dataset(X_train, y_train)
X_val_img, y_val = create_dataset(X_val, y_val)
X_test_img, y_test = create_dataset(X_test, y_test)

# Build model
model_x_original = build_xception(num_classes)

# Train
model_x_original.fit(
    X_train_img, y_train,
    validation_data=(X_val_img, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE
)

# Evaluate
loss3, acc3 = model_x_original.evaluate(X_test_img, y_test)

print("\nXception trained on ORIGINAL labels Accuracy:", acc3)


Epoch 1/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 345s 2s/step - accuracy: 0.5790 - loss: 0.9300 - val_accuracy: 0.4823 - val_loss: 1.2893
Epoch 2/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 326s 2s/step - accuracy: 0.7138 - loss: 0.6832 - val_accuracy: 0.4823 - val_loss: 1.1226
Epoch 3/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 325s 2s/step - accuracy: 0.8147 - loss: 0.4635 - val_accuracy: 0.4823 - val_loss: 1.9240
Epoch 4/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 327s 2s/step - accuracy: 0.8888 - loss: 0.2921 - val_accuracy: 0.6760 - val_loss: 1.0719
Epoch 5/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 326s 2s/step - accuracy: 0.9406 - loss: 0.1536 - val_accuracy: 0.6562 - val_loss: 1.4193
Epoch 6/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 326s 2s/step - accuracy: 0.9478 - loss: 0.1350 - val_accuracy: 0.2240 - val_loss: 7.9630
Epoch 7/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 326s 2s/step - accuracy: 0.9632 - loss: 0.1076 - val_accuracy: 0.8531 - val_loss: 0.6617
Epoch 8/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 327s 2s/step - accuracy: 0.9708 - loss: 0.0840 - val_accu

In [23]:
# Prepare data (Birch cluster labels)

X_train, X_val, X_test, y_train, y_val, y_test, num_classes = prepare_data(
    xcep_birch_df,
    label_column="cluster_label"   # <-- change if different
)

X_train_img, y_train = create_dataset(X_train, y_train)
X_val_img, y_val = create_dataset(X_val, y_val)
X_test_img, y_test = create_dataset(X_test, y_test)

# Build model
model_x_cluster = build_xception(num_classes)

# Train
model_x_cluster.fit(
    X_train_img, y_train,
    validation_data=(X_val_img, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE
)

# Evaluate
loss4, acc4 = model_x_cluster.evaluate(X_test_img, y_test)

print("\nXception trained on Birch labels Accuracy:", acc4)


Epoch 1/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 346s 2s/step - accuracy: 0.7748 - loss: 0.5313 - val_accuracy: 0.0979 - val_loss: 1.2290
Epoch 2/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 328s 2s/step - accuracy: 0.8520 - loss: 0.3482 - val_accuracy: 0.0979 - val_loss: 1.9323
Epoch 3/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 328s 2s/step - accuracy: 0.8725 - loss: 0.2972 - val_accuracy: 0.0979 - val_loss: 4.4404
Epoch 4/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 327s 2s/step - accuracy: 0.8886 - loss: 0.2734 - val_accuracy: 0.6906 - val_loss: 0.7595
Epoch 5/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 327s 2s/step - accuracy: 0.9076 - loss: 0.2273 - val_accuracy: 0.9042 - val_loss: 0.2128
Epoch 6/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 327s 2s/step - accuracy: 0.9076 - loss: 0.2204 - val_accuracy: 0.8896 - val_loss: 0.2704
Epoch 7/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 327s 2s/step - accuracy: 0.9301 - loss: 0.1753 - val_accuracy: 0.6562 - val_loss: 1.6542
Epoch 8/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 328s 2s/step - accuracy: 0.9335 - loss: 0.1709 - val_accu

In [24]:

# =========================
# FINAL COMPARISON
# =========================

print("\n========== FINAL RESULTS ==========")
print("EfficientNet Original Accuracy:", acc1)
print("EfficientNet Cluster Accuracy :", acc2)
print("Xception Original Accuracy    :", acc3)
print("Xception Cluster Accuracy     :", acc4)


========== FINAL RESULTS ==========
EfficientNet Original Accuracy: 0.9114583134651184
EfficientNet Cluster Accuracy : 0.903124988079071
Xception Original Accuracy    : 0.19687500596046448
Xception Cluster Accuracy     : 0.40416666865348816
